In [1]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [2]:
cd ml-100k/

/Users/ayalahlou/Final Project 001/ml-100k


In [3]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)

In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

In [5]:
def split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size=10, replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    return train, test
train, test = split(ratings)

In [6]:
mov=pd.read_csv('movies_metadata.csv')
mov=mov.loc[:,'imdb_id':'original_title']
mov=mov.drop(columns=['original_language'])

## DEMO 1: COSINE SIMILARITY

In [7]:
def cosCF():
    fav=input("What is your favourite movie ? ")
    try:
        def similarityfunc(ratings, kind='user', epsilon=1e-9):
            if kind == 'user':
                sim = ratings.dot(ratings.T) + epsilon
            elif kind == 'item':
                sim = ratings.T.dot(ratings) + epsilon
            norms = np.array([np.sqrt(np.diagonal(sim))])
            return (sim / norms / norms.T)

        user_similarity = similarityfunc(train, kind='user')
        item_similarity = similarityfunc(train, kind='item')

        def predict_topk(ratings, similarity, kind='user', k=40):
            pred = np.zeros(ratings.shape)
            if kind == 'user':
                for i in range(ratings.shape[0]):
                    top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
                    for j in range(ratings.shape[1]):
                        pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                        pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
            if kind == 'item':
                for j in range(ratings.shape[1]):
                    top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
                    for i in range(ratings.shape[0]):
                        pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                        pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items]))        

            return pred

        idx_to_movie = {}
        a=[]
        with open('u.item', 'r',errors='replace') as f:
            for line in f.readlines():
                info = line.split('|')
                info[1]= info[1].split(" (")[0]
                idx_to_movie[int(info[0])-1] = info[1]
                a.append(info)
        def top_k_movies(similarity, mapper, movie_idx, k=11):
            return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]

        for i in range(1682):
            if a[i][1]==fav:
                idx=i
                #print('hi')
                break
        movies = top_k_movies(item_similarity, idx_to_movie, idx)
        li=[]


        for j in range(len(movies)):
            for i in range(len(mov)):
                if movies[j]==mov['original_title'][i]:
                    reclink='https://www.imdb.com/title/'+mov['imdb_id'][i]+'/?ref_=fn_al_tt_1'
                    li.append(reclink)
                    break
                elif movies[j]!=mov['original_title'][i] and i==45465:
                    reclink='(link not available)'
                    li.append(reclink)
                    break   
        print('\n Users who liked', fav ,'also liked: \n \n *',movies[1], li[1],'\n','*',movies[2], li[2],'\n','*',movies[3], li[3],'\n','*',movies[4], li[4],'\n','*',movies[5], li[5],'\n','*',movies[6], li[6],'\n','*',movies[7], li[7],'\n','*',movies[8], li[8],'\n','*',movies[9], li[9],'\n','*',movies[10], li[10])
    except:
        print ("It seems like we couldn't find your favourite movie. Try again using another movie!")


In [8]:
cosCF()

What is your favourite movie ? Toy Story

 Users who liked Toy Story also liked: 
 
 * Star Wars https://www.imdb.com/title/tt0076759/?ref_=fn_al_tt_1 
 * Return of the Jedi https://www.imdb.com/title/tt0086190/?ref_=fn_al_tt_1 
 * Independence Day https://www.imdb.com/title/tt0116629/?ref_=fn_al_tt_1 
 * Fargo https://www.imdb.com/title/tt0116282/?ref_=fn_al_tt_1 
 * Willy Wonka and the Chocolate Factory (link not available) 
 * Raiders of the Lost Ark https://www.imdb.com/title/tt0082971/?ref_=fn_al_tt_1 
 * Star Trek: First Contact https://www.imdb.com/title/tt0117731/?ref_=fn_al_tt_1 
 * Rock, The (link not available) 
 * Mission: Impossible https://www.imdb.com/title/tt0117060/?ref_=fn_al_tt_1 
 * Jerry Maguire https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1


In [12]:
def PCCF():
    fav=input("What is your favourite movie ? ")
    try:
        from sklearn.metrics import pairwise_distances

        item_correlation = 1 - pairwise_distances(train.T, metric='correlation')
        item_correlation[np.isnan(item_correlation)] = 0.

        idx_to_movie = {}
        a=[]
        with open('u.item', 'r',errors='replace') as f:
            for line in f.readlines():
                info = line.split('|')
                info[1]= info[1].split(" (")[0]
                idx_to_movie[int(info[0])-1] = info[1]
                a.append(info)

        def top_k_movies(similarity, mapper, movie_idx, k=11):
            return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]
        for i in range(1682):
            if a[i][1]==fav:
                idx=i
                break
        movies = top_k_movies(item_correlation, idx_to_movie, idx)
        #print(movies)
        li=[]
        for j in range(len(movies)):
            for i in range(len(mov)):
                if movies[j]==mov['original_title'][i]:
                    reclink='https://www.imdb.com/title/'+mov['imdb_id'][i]+'/?ref_=fn_al_tt_1'
                    li.append(reclink)
                    break
                elif movies[j]!=mov['original_title'][i] and i==45465:
                    reclink='(link not available)'
                    li.append(reclink)
                    break   
        print('\n Users who liked', fav ,'also liked: \n \n *',movies[1], li[1],'\n','*',movies[2], li[2],'\n','*',movies[3], li[3],'\n','*',movies[4], li[4],'\n','*',movies[5], li[5],'\n','*',movies[6], li[6],'\n','*',movies[7], li[7],'\n','*',movies[8], li[8],'\n','*',movies[9], li[9],'\n','*',movies[10], li[10])
    except:
        print ("It seems like we couldn't find your favourite movie ☉_☉ . Try again!")


In [13]:
PCCF() 

What is your favourite movie ? Toy Story

 Users who liked Toy Story also liked: 
 
 * Willy Wonka and the Chocolate Factory (link not available) 
 * Independence Day https://www.imdb.com/title/tt0116629/?ref_=fn_al_tt_1 
 * Star Wars https://www.imdb.com/title/tt0076759/?ref_=fn_al_tt_1 
 * Return of the Jedi https://www.imdb.com/title/tt0086190/?ref_=fn_al_tt_1 
 * Star Trek: First Contact https://www.imdb.com/title/tt0117731/?ref_=fn_al_tt_1 
 * Rock, The (link not available) 
 * Beauty and the Beast https://www.imdb.com/title/tt0101414/?ref_=fn_al_tt_1 
 * Mission: Impossible https://www.imdb.com/title/tt0117060/?ref_=fn_al_tt_1 
 * Birdcage, The (link not available) 
 * Aladdin https://www.imdb.com/title/tt0103639/?ref_=fn_al_tt_1


In [ ]:
##everything is working fine, I should probably include the ratings of each film. 

In [14]:
dkucg=pd.read_csv('dkucinemaG.csv')
dkucg

,Genre,Title,Audience
0,Drama,Gifted,35
1,Drama,Flipped,92
2,Drama,Interstellar,21
3,Drama,The Godfather,34
4,Drama,The Pursuit of Happyness,27
5,Drama,10 Things I Hate About You,36
6,Action,2012,16
7,Romance,Flipped,92
8,Romance,10 Things I Hate About You,27
9,Adventure,Interstellar,21


In [15]:
DKUM=dkucg.groupby(['Title']).mean().sort_values(by=['Audience'], ascending=False)
mostpop=DKUM['Audience'].index.values[0]
DKUM['li']=[1,2,3,4,5,6,7,8]
DKUM

,Audience,li
Title,,
Flipped,92,1
Gifted,35,2
The Godfather,34,3
10 Things I Hate About You,30,4
The Pursuit of Happyness,27,5
Interstellar,21,6
Interstellar,21,7
2012,16,8


In [ ]:
PCCF()
